In [1]:
## program for the search function 
import numpy as np
import pandas as pd
from os.path import join as oj
import os
import pandas as pd
import sys
import inspect
from datetime import datetime, timedelta
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
sys.path.append(parentdir + '/modeling')
import load_data
from fit_and_predict import add_preds
from functions import merge_data
from viz import  viz_interactive, viz_static, viz_map_utils
import plotly.figure_factory as ff
import plotly
import re


#Load data and add prediction
def add_pre(df, var, name):
    h = df_county[var + '1-day'].copy()
    for i in range(2,8):
        h = np.vstack((h,df_county[var + str(i) +'-day']))
    df_county[name] = [h.T[i] for i in range(len(h.T))]


# generate html for individual counties
def generate_all_counties():
    print('generating html for counties')
    df_tab = df_county[['CountyName', 'StateName', 
             'deaths', 'cases','countyFIPS','pred_cases','pred_deaths','Predicted Deaths Intervals','Predicted Cases Intervals']]
    df_tab = df_tab.rename(columns={'CountyName': 'County', 'StateName': 'State',
                                'Predicted Deaths Intervals': 'pred_deaths_interval',
                                'Predicted Cases Intervals': 'pred_cases_interval'})
    dates = viz_map_utils.date_in_data(df_county)
    viz_interactive.viz_curves_all_counties(df_tab, oj(parentdir, 'results/All_counties/'), dates)
    print('succesfully generated all county html')

# Generate map plot 
def generate_map():
    print('generating search map')
    fips = df_county['countyFIPS']
    values = df_county['tot_cases']
    binning = []
    level = 6
    for i in range(level):
        binning.append(np.rint(2**(np.log2(max(values))/(level+1)*(i+1))))
    fig = ff.create_choropleth(fips=fips, values=values,
                           colorscale = ['#F7E8E4','#F5C8BB','#B96D67','#A83C3B',
                                '#8B2222','#5B0D0D','#5A2318'],binning_endpoints=binning,
                           legend_title='Cases'
                          )

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,255)',
        plot_bgcolor='rgba(0,0,0,255)',
        template='plotly_dark'
    )
    fig['layout'].update(width=1000, height=500, autosize=True)

    fig.write_image("results/search_map.png",width=900, height=450)
    
    print('succesfully generated search map')   
    
    return plotly.offline.plot(fig,
                include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js',
                   output_type='div')


# update the html file (conbine search1 + search2 + search3)
def update_html(search2):
    id =re.search('<div id="([^ ]*)"',search2)
    print(id,id.group(1),id.group(0))
    with open('results/search1.html', 'r') as content_file:
        search1 = content_file.read()
    with open('results/search3.html', 'r') as content_file:
        search3 = content_file.read()
    f = open('results/search.html',"w+")
    f.write(search1+search2+search3.replace("map id to replace",id.group(1)))
    print('succesfully updated search html')



Loading BokehJS ...

In [2]:
if __name__ == '__main__':
    print('loading data...')
    NUM_DAYS_LIST = [1, 2, 3, 4, 5, 6, 7]
    df_county = load_data.load_county_level(data_dir=oj(parentdir, 'data')).fillna(0)
    df_county = add_preds(df_county, NUM_DAYS_LIST=NUM_DAYS_LIST, cached_dir=oj(parentdir, 'data')) # adds keys like "Predicted Deaths 1-day"

    ## orgnize predicts as array
    add_pre(df_county,'Predicted Cases ','pred_cases')
    add_pre(df_county,'Predicted Deaths ','pred_deaths')

    #generate_all_counties()
    #search2 = generate_map()
    #update_html(search2)

loading data...
loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [3]:
df_tab = df_county[['CountyName', 'StateName', 
             'deaths', 'cases','countyFIPS','pred_cases','pred_deaths','Predicted Deaths Intervals','Predicted Cases Intervals']]
df_tab = df_tab.rename(columns={'CountyName': 'County', 'StateName': 'State',
                                'Predicted Deaths Intervals': 'pred_deaths_interval',
                                'Predicted Cases Intervals': 'pred_cases_interval'})
dates = viz_map_utils.date_in_data(df_county)
date1 =dates
#viz_interactive.viz_curves_all_counties(df_tab, parentdir, dates)

In [4]:
print(parentdir)

/usr/local/google/home/danqingwang/covid19-severity-prediction


In [5]:
from bokeh.sampledata import us_states, us_counties
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh import palettes
from bokeh.models import ColorBar,HoverTool,LinearColorMapper,ColumnDataSource,FixedTicker, LogColorMapper
output_notebook()
import re
import numpy as np
from modeling import fit_and_predict
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import plot
from plotly.subplots import make_subplots
import json
import plotly.express as px
import plotly
import pandas as pd
from datetime import datetime, timedelta

row = df_tab.iloc[0]
state,county,FIPS = row['State'],row['County']+" County",row['countyFIPS']
newdates = [dates[-1] + timedelta(days =i) for i in range(0,8)]
fig = make_subplots(rows=1, cols=2, specs=[[{"secondary_y": True},{"secondary_y": True}]])
traces = []
keys_curves=['deaths','cases']
credstr ='rgb(179,0,10)'
cbluestr = 'rgb(0,0,179)'
fill_strings = ['rgba(179,0,10,0.4)','rgba(0,0,179,0.4)']
color_strings = [credstr, cbluestr]




x = [d.strftime("%m/%d") for d in dates]
for j, key_curve in enumerate(keys_curves):
        curve = row[key_curve]
        traces = []
        traces.append(go.Scatter(x=dates,
                    y=curve,
                    showlegend=True,
                    visible=True, #key == key0,# False, #key == key0,
                    name=key_curve,
                    line=dict(color=color_strings[j], width=4))        )
        low = np.hstack((curve[-1],[x[0] for x in row['pred_'+key_curve+'_interval']]))
        up = np.hstack((curve[-1],[x[1] for x in row['pred_'+key_curve+'_interval']]))
        traces.append(go.Scatter(
                    name = 'Upper Bound',
                    showlegend= False,
                    x = newdates,
                    y = up,
                    line=dict(dash ='solid',color=fill_strings[j], width=4)))
        traces.append(go.Scatter(x= newdates,
                    y=np.hstack(([curve[-1]],row['pred_'+key_curve])),
                    showlegend= False,
                    visible=True,
                    mode = 'lines',
                    name = 'prediction on '+key_curve,
                    line=dict(dash ='dash',color=color_strings[j], width=4),
                    fillcolor = fill_strings[j],
                    fill ='tonexty'))
        traces.append(go.Scatter(
                    name = 'Lower Bound',
                    visible = True,
                    showlegend= False,
                    x = newdates,
                    y = low,
                    fill ='tonexty',
                    fillcolor = fill_strings[j],
                    line=dict(dash ='solid',color=fill_strings[j], width=4)))
        for trace in traces:
            fig.add_trace(trace, secondary_y=key_curve=="deaths", row=1,col=1)
            
        
        # initialize xaxis2 and yaxis2
fig['layout']['xaxis'] = {}
fig['layout']['yaxis'] = {}
        
        
        # Edit layout for subplots
#fig.layout.xaxis.update({'domain': [0.05, .95]})
fig.layout.xaxis.update({'title': 'Date'})

y1 = max([x[1] for x in row['pred_cases_interval']])
y2 = max([x[1] for x in row['pred_deaths_interval']])

fig.layout.yaxis.update({'domain': [0, .95]})
fig.layout.yaxis.update({'title': 'Cases','rangemode':'tozero','dtick': y1/5},range=[0,y1])
fig.update_yaxes(title_text="Deaths",rangemode = 'tozero',dtick = y2*1.3/5, range=[0,y2*1.3], secondary_y=True)


        # Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':150, 'b':120})
        
fig.layout.update(
            title ={'text':state+' '+county+ '<br>' + 'FIPS:'+row['countyFIPS'], 'y':0.93,
        'x':0.05, 
        'yanchor': 'top'},
        font=dict(
        size=19))


fig.update_layout(xaxis_range=[datetime(2020, 3, 25),
                               newdates[-1]])
fig.layout.template = 'plotly_dark'
plotly.offline.plot(fig, filename='test.html', auto_open=True)
print(row)

Loading BokehJS ...

County                                                              Kings
State                                                                  NY
deaths                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
cases                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
countyFIPS                                                          36047
pred_cases              [58837.296072178506, 58996.86029877838, 59105....
pred_deaths             [6992.078677664294, 7007.1518277604855, 7022.4...
pred_deaths_interval    [(6977, 7007.324666177754), (6977.910517205515...
pred_cases_interval     [(58634, 58773.509668105486), (58659.860382318...
Name: 1843, dtype: object


In [6]:
df_tab

,County,State,deaths,cases,countyFIPS,pred_cases,pred_deaths,pred_deaths_interval,pred_cases_interval
1843,Kings,NY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36047,"[58837.296072178506, 58996.86029877838, 59105....","[6992.078677664294, 7007.1518277604855, 7022.4...","[(6977, 7007.324666177754), (6977.910517205515...","[(58634, 58773.509668105486), (58659.860382318..."
1860,Queens,NY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36081,"[64566.57212713107, 64724.024158870016, 64853....","[6495.948715026794, 6510.715820806619, 6525.67...","[(6487, 6511.632288736748), (6487, 6541.655512...","[(64407.50464671045, 64570.79050282633), (6444..."
1822,Bronx,NY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36005,"[47024.066062495855, 47127.4684496434, 47219.0...","[4636.665299789166, 4647.299227411613, 4658.18...","[(4628, 4649.739725271404), (4628, 4667.220926...","[(46926.840424121336, 47048.13523115705), (469..."
602,Cook,IL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, ...",17031,"[86871.50002149612, 87207.50004432307, 87543.5...","[4409.5432945903685, 4452.47089115127, 4495.71...","[(4378.420429376475, 4438.27282733497), (4375....","[(86836.35621791743, 86906.64381798217), (8715..."
199,Los Angeles,CA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",06037,"[81180.50113831054, 82641.80248587231, 84103.1...","[3104.0733158847156, 3146.2309791212956, 3188....","[(3074.7366080002485, 3130.5006118559677), (30...","[(79875.72337864758, 82485.27862764377), (8090..."
...,...,...,...,...,...,...,...,...,...
1874,Tompkins,NY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36109,"[174.10442280413213, 174.78278423094056, 174.9...","[0.153246986680331, 0.2965474762746128, 0.4320...","[(0.0, 0.2991872438968294), (0.0, 0.5776960870...","[(173, 173.57690416602128), (173, 173.70982563..."
662,Pope,IL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",17151,"[1.2586225731252956, 1.5259270655463324, 1.804...","[0.14570182134189313, 0.2993152400738649, 0.46...","[(0.0, 0.29412796827363913), (0.0, 0.608578382...","[(1, 1.5012667550859005), (1, 2.01290007827575..."
663,Pulaski,IL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",17153,"[66.13663561122854, 66.88522998099634, 67.4805...","[0.15200351673772372, 0.29609438812136324, 0.4...","[(0.0, 0.29954899533872603), (0.0, 0.584167370...","[(65, 68.11414509266243), (65, 70.035299697488..."
664,Putnam,IL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",17155,"[1.2529888628169923, 1.5025454245508176, 1.750...","[0.1490887085684377, 0.2991643349391045, 0.454...","[(0.0, 0.30076368661646596), (0.0, 0.607091470...","[(1, 1.4914459504201096), (1, 1.97093259747317..."


In [7]:
def add_predictions_7day(data,df):
    data = data.sort_values(by='countyFIPS')
    dic = {'cases':'Cases','deaths':'Deaths'}
    for i in range(df.shape[0]):
        for key in ['cases','deaths']:
            df.loc[i,'pred_7day_'+key].append(data.loc[i,'Predicted '+ dic[key] +' 7-day'])
            df.loc[i,'pred_7day_'+key+'_interval'].append(data.loc[i,'Predicted '+dic[key]+' Intervals'][6])
cached_dir=oj(parentdir, 'data')
i = 0
for c in ['deaths','cases']:
    df_tab['pred_7day_'+c] =[[] for _ in range(df_tab.shape[0])]
    df_tab['pred_7day_'+c+'_interval'] =[[] for _ in range(df_tab.shape[0])]
df_tab = df_tab.sort_values(by='countyFIPS')
date2 = []
k = 0
while True:
    d = (datetime.today() - timedelta(days=i)).date()
    i += 1
    if cached_dir is not None:
        cached_fname = oj(cached_dir, f'preds_{d.month}_{d.day}_cached.pkl')
        if os.path.exists(cached_fname):
            date2.append(d+timedelta(days=7))
            add_predictions_7day(pd.read_pickle(cached_fname),df_tab)
        else:
            k += 1
            if k > 1:
                break

In [65]:
row = df_tab.iloc[0]
state,county,FIPS = row['State'],row['County']+" County",row['countyFIPS']
newdates = date1
fig = make_subplots(rows=1, cols=2, specs=[[{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=("Time series with 7 day prediction",
                                    "Time series with historical prediction(7 day)"))
traces = []
keys_curves=['deaths','cases']
red = '179,74,71'
blue = '83,137,190'
credstr ='rgb(' + red + ')'
cbluestr = 'rgb(' + blue + ')'
fill_strings = ['rgba(' + red + ',0.4)','rgba(' + blue + ',0.4)']
color_strings=[credstr,cbluestr]

def cal_coverage(row,date1,date2,keys = ['deaths', 'cases']):
    def cal(key, key_inv):
        dic_real = {}
        dic_pre = {}
        for i in range(len(date1)):
            dic_real[date1[i]] = row[key][i]
        for i in range(len(date2)):
            dic_pre[date2[i]] = row[key_inv][i]
        hit = miss = 0
        for d in dic_pre.keys():
            if d in dic_real:
                if dic_real[d] >= round(dic_pre[d][0],0) and dic_real[d] <= round(dic_pre[d][1],0):
                    hit += 1
                else:
                    miss += 1
        return (hit/(hit+miss))
    return [cal(key, 'pred_7day_' + key +'_interval') for key in keys]
def make_traces(fig, row, pre, date1, date2, r, c, width,show_lengend,keys_curves=['deaths','cases']):
    for j, key_curve in enumerate(keys_curves):
        curve = row[key_curve]
        traces = []
        traces.append(go.Scatter(x=date1,
                    y=curve,
                    showlegend=show_lengend,
                    visible=True, #key == key0,# False, #key == key0,
                    name=key_curve,
                    line=dict(color=color_strings[j], width=width))        )
        low = np.hstack((curve[-1],[x[0] for x in row[pre+key_curve+'_interval']]))
        up = np.hstack((curve[-1],[x[1] for x in row[pre+key_curve+'_interval']]))
        if pre == 'pred_7day_':
            low = low[1:]
            up = up[1:]
        traces.append(go.Scatter(
                    name = 'Upper Bound',
                    showlegend= False,
                    x = date2,
                    y = up,
                    marker = dict(size=0),
                    line=dict(dash ='solid',color=fill_strings[j], width=0)))
        traces.append(go.Scatter(x= date2,
                    y=row[pre+key_curve],
                    name = key_curve +' prediction',
                    showlegend= show_lengend,
                    mode = 'lines',
                    line=dict(dash ='dash',color=color_strings[j], width=width),
                    fillcolor = fill_strings[j],
                    fill ='tonexty'))
        traces.append(go.Scatter(
                    name = 'Lower Bound',
                    showlegend= False,
                    marker = dict(size=0),
                    x = date2,
                    y = low,
                    fill ='tonexty',
                    fillcolor = fill_strings[j],
                    line=dict(dash ='solid',color=fill_strings[j], width=0)))
        for trace in traces:
            fig.add_trace(trace, secondary_y=key_curve=="deaths", row=r,col=c)
    return fig
fig = make_traces(fig, row,'pred_7day_',date1,date2,1,2,3,False)
newdates = [date1[-1] + timedelta(days =i) for i in range(0,8)]
fig = make_traces(fig, row,'pred_',date1,newdates,1,1,4,True)            
for i in fig['layout']['annotations']:
    i['font'] = dict(size=15,color='white')  

deaths_cov,cases_cov = cal_coverage(row,date1,date2)
# Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':150, 'b':120})
        
fig.layout.update(
            title = dict(text = county+', '+state+ '<br>' + 'FIPS:'+row['countyFIPS'], 
                    y = 0.92,
                    x = 0.03, 
                    yanchor = 'top'),
            width=1200,height=700,
            font=dict(size=12),
            legend=dict(x=0.5, y=1.2),
            legend_orientation="h"
                )
fig.add_annotation(dict(font=dict(color="white",size=11),
                            x=0.8,
                            y=0.9,
                            showarrow=False,
                            text='Cases Prediction coverage: '+str(round(cases_cov, 2))+'<br>'
                                '  Deaths Prediction coverage: '+str(round(deaths_cov, 2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"
                           ))
fig.add_annotation(dict(font=dict(color="white",size=12),
                            x=0.9,
                            y=1.3,
                            showarrow=False,
                            text='Modebar '+u'\u2191',
                            textangle=0,
                            xref="paper",
                            yref="paper"
                           ))
## Set plot1 axes properties
y1 = max([x[1] for x in row['pred_cases_interval']])
y2 = max([x[1] for x in row['pred_deaths_interval']])
fig.update_xaxes(title_text="Date",range=[datetime(2020, 3, 24), newdates[-1]],domain = [0,0.41],row=1, col=1)
fig.update_yaxes(title_text="Cases", color=cbluestr,rangemode = 'tozero',
            dtick = (y1-y11)/5,range=[y11,y1], domain=[0,0.95],row=1, col=1)  
fig.update_yaxes(title_text="Deaths",showgrid=False,rangemode = 'tozero',color=credstr,
            dtick = (y2*1.3-y22)/5, range=[y22,y2*1.3], secondary_y=True, row=1,col=1)
        ## Set plot2 axes properties
fig.update_xaxes(title_text="Date",range=[date2[-1], date2[7]],domain = [0.59,1], row=1, col=2)
fig.update_yaxes(title_text="Cases", color=cbluestr,rangemode = 'tozero',
            dtick = (y1-y11)/5,range=[y11,y1], domain=[0,0.95],row=1, col=2)  
fig.update_yaxes(title_text="Deaths",rangemode = 'tozero',color=credstr,
            dtick = (y2-y22)*1.3/5, showgrid=False,range=[y22,y2*1.3], secondary_y=True, row=1,col=2)
fig.layout.template = 'plotly_dark'
plotly.offline.plot(fig, filename='test.html', auto_open=True)

'test.html'

In [ ]:
print(len(date2))
len(row['pred_7day_cases_interval'])

In [18]:
f'{10.324335:03d}'

ValueError: Unknown format code 'd' for object of type 'float'

In [27]:
u'\u2191'

'↑'